<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 10. Бустинг
## <center>Часть 9. Xgboost и несбалансированные выборки

## Общие советы
Есть несколько общих советов по работе с несбалансированными выборками:

- собрать больше данных
- использовать метрики, нечувствительные к дисбалансу классов (F1, ROC AUC)
- oversampling/undersampling - брать больше объектов мало представленного класса, и мало - частого класса
- создать искусственные объекты, похожие на объекты редкого класса (например, алгоритмом SMOTE)

С XGBoost можно:
- следить за тем, чтобы параметр `min_child_weight` был  мал, хотя по умолчанию он и так равен 1.
- задать ббольше веса некоторым объектам при инициализации `DMatrix`
- контролировать отшошение числа представителей разных классов с помощью параметра `set_pos_weight`

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

**Сгенерируем несбалансированную выборку для задачи классификации.**

In [2]:
X, y = make_classification(
    n_samples=200,
    n_features=5,
    n_informative=3,
    n_classes=2,
    weights=[0.9, 0.1],
    shuffle=True,
    random_state=123,
)

print("There are {} positive instances.".format(y.sum()))

There are 20 positive instances.


**Разбиваем на обучающую и тестовую выборки. Соблюдаем стратификацию.**

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, stratify=y, random_state=123
)

print("Train set labels distibution: {}".format(np.bincount(y_train)))
print("Test set labels distibution:  {}".format(np.bincount(y_test)))

Train set labels distibution: [121  13]
Test set labels distibution:  [59  7]


**В начале игнорируем то, что выборка несбалансированная.**

In [4]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

**Инициализируем параметры Xgboost - будем обучать композицию из 15 "пеньков".**

In [5]:
params = {"objective": "binary:logistic", "max_depth": 1, "silent": 1, "eta": 1}

num_rounds = 15

In [6]:
xgb_model = xgb.train(params, dtrain, num_rounds)
y_test_preds = (xgb_model.predict(dtest) > 0.5).astype("int")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:56:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


**Матрица ошибок.**

In [7]:
pd.crosstab(
    pd.Series(y_test, name="Actual"),
    pd.Series(y_test_preds, name="Predicted"),
    margins=True,
)

Predicted,0,1,All
Actual,,,
0,58,1,59
1,5,2,7
All,63,3,66


**Доля правильных ответов, точность и полнота.**

In [8]:
print("Accuracy: {0:.2f}".format(accuracy_score(y_test, y_test_preds)))
print("Precision: {0:.2f}".format(precision_score(y_test, y_test_preds)))
print("Recall: {0:.2f}".format(recall_score(y_test, y_test_preds)))

Accuracy: 0.91
Precision: 0.67
Recall: 0.29


**Видно, что полнота низкая. то есть алгоритм плохо распознает объекты мало представленного класса. Если интересно находить как раз такие редкие объекты, то от такого алгоритма мало толку.**

## Задание весов вручную
**При создании объекта `DMatrix` можно сразу явно указать, что вес положительных объектов в 5 раз больше, чем отрицательных.**

In [9]:
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 5

dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights)  # weights added
dtest = xgb.DMatrix(X_test)

**Повторим обучение модели, как и в предыдущем случае.**

In [10]:
xgb_model = xgb.train(params, dtrain, num_rounds)
y_test_preds = (xgb_model.predict(dtest) > 0.5).astype("int")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:58:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [11]:
pd.crosstab(
    pd.Series(y_test, name="Actual"),
    pd.Series(y_test_preds, name="Predicted"),
    margins=True,
)

Predicted,0,1,All
Actual,,,
0,57,2,59
1,4,3,7
All,61,5,66


In [12]:
print("Accuracy: {0:.2f}".format(accuracy_score(y_test, y_test_preds)))
print("Precision: {0:.2f}".format(precision_score(y_test, y_test_preds)))
print("Recall: {0:.2f}".format(recall_score(y_test, y_test_preds)))

Accuracy: 0.91
Precision: 0.60
Recall: 0.43


**Видим, что вес объектов надо настраивать в зависимости от задачи.**

## Параметр `scale_pos_weight` в Xgboost
**Задание весов вручную можно заменить на параметр `scale_pos_weight`**

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

**Инициализируем параметр `scale_pos_weight` соотношением числа отрицательных и положительных объектов.**

In [14]:
train_labels = dtrain.get_label()

ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1)
params["scale_pos_weight"] = ratio

In [15]:
xgb_model = xgb.train(params, dtrain, num_rounds)
y_test_preds = (xgb_model.predict(dtest) > 0.5).astype("int")

pd.crosstab(
    pd.Series(y_test, name="Actual"),
    pd.Series(y_test_preds, name="Predicted"),
    margins=True,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:59:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Predicted,0,1,All
Actual,,,
0,56,3,59
1,4,3,7
All,60,6,66


In [16]:
print("Accuracy: {0:.2f}".format(accuracy_score(y_test, y_test_preds)))
print("Precision: {0:.2f}".format(precision_score(y_test, y_test_preds)))
print("Recall: {0:.2f}".format(recall_score(y_test, y_test_preds)))

Accuracy: 0.89
Precision: 0.50
Recall: 0.43


**В этом случае значение параметра `scale_pos_weight` надо выбирать в зависимости от желаемого соотношения между точностью и полнотой.**

## Пример с оттоком клиентов телеком-компании

**Загрузим данные и осуществим минимальную предобработку.**

In [18]:
# df = pd.read_csv("../../data/telecom_churn.csv")
df = pd.read_csv('https://github.com/Yorko/mlcourse.ai/raw/main/data/telecom_churn.csv')

In [19]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


**Штаты просто занумеруем, а признаки International plan (наличие международного роуминга), Voice mail plan (наличие голосовой почтыы) и целевой Churn сделаем бинарными.**

In [20]:
state_enc = LabelEncoder()
df["State"] = state_enc.fit_transform(df["State"])
df["International plan"] = (df["International plan"] == "Yes").astype("int")
df["Voice mail plan"] = (df["Voice mail plan"] == "Yes").astype("int")
df["Churn"] = (df["Churn"]).astype("int")

**Видим, что соотношение хороших и плохих клиентов примерно 6:1.**

In [21]:
df["Churn"].value_counts()

Churn
0    2850
1     483
Name: count, dtype: int64

**Разделим данные на обучающую и тестовую выборки в отношении 7:3 с учетом соотношения классов. Инициализируем соотв. объекты DMatrix dtrain и dtest.**

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Churn", axis=1),
    df["Churn"],
    test_size=0.3,
    stratify=df["Churn"],
    random_state=42,
)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [23]:
params = {"objective": "binary:logistic", "max_depth": 4, "silent": 1, "eta": 0.3}

num_rounds = 100

In [24]:
xgb_model = xgb.train(params, dtrain, num_rounds)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:00:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [25]:
preds_prob = xgb_model.predict(dtest)
pred_labels = (preds_prob > 0.5).astype("int")

In [27]:
pd.crosstab(
    pd.Series(dtest.get_label(), name="Actual"),
    pd.Series(pred_labels, name="Predicted"),
    margins=True,
)

Predicted,0,1,All
Actual,,,
0.0,845,10,855
1.0,34,111,145
All,879,121,1000


In [28]:
print("Accuracy: {0:.2f}".format(accuracy_score(dtest.get_label(), pred_labels)))
print("Precision: {0:.2f}".format(precision_score(dtest.get_label(), pred_labels)))
print("Recall: {0:.2f}".format(recall_score(dtest.get_label(), pred_labels)))
print("F1: {0:.2f}".format(f1_score(dtest.get_label(), pred_labels)))

Accuracy: 0.96
Precision: 0.92
Recall: 0.77
F1: 0.83


**Теперь изменим параметр `scale_pos_weight` и проделаем то же самое.**

In [29]:
params["scale_pos_weight"] = 10

In [30]:
xgb_model = xgb.train(params, dtrain, num_rounds)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:01:07] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [31]:
preds_prob = xgb_model.predict(dtest)
pred_labels = (preds_prob > 0.5).astype("int")

In [32]:
pd.crosstab(
    pd.Series(dtest.get_label(), name="Actual"),
    pd.Series(pred_labels, name="Predicted"),
    margins=True,
)

Predicted,0,1,All
Actual,,,
0.0,827,28,855
1.0,28,117,145
All,855,145,1000


In [33]:
print("Accuracy: {0:.2f}".format(accuracy_score(dtest.get_label(), pred_labels)))
print("Precision: {0:.2f}".format(precision_score(dtest.get_label(), pred_labels)))
print("Recall: {0:.2f}".format(recall_score(dtest.get_label(), pred_labels)))
print("F1: {0:.2f}".format(f1_score(dtest.get_label(), pred_labels)))

Accuracy: 0.94
Precision: 0.81
Recall: 0.81
F1: 0.81


**Видим, что таким образом мы настроили модель так, что она меньше ошибается в распознавании плохих клиентов.**